# HW 5-2 Regression
20220041

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

In [69]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_feature = train_data.drop(columns=['MIN'])
train_label = train_data['MIN']

extra_cols = set(train_feature.columns) - set(test_data.columns)
train_data = train_data.drop(columns=extra_cols)

print("Training features:", train_feature.head())
print("Training label:", train_label.head())

Training features:   SEASON_ID     TEAM_ID  PLAYER_AGE  FGM  FGA  ...  BLK   TOV   PF  PTS  position
0   2004-05  1610612740        34.0  133  342  ...  1.0  54.0   65  361     Guard
1   2001-02  1610612746        26.0   13   39  ...  6.0  10.0   16   54    Center
2   1964-65  1610612765        28.0   84  221  ...  NaN   NaN  136  208    Center
3   1988-89  1610612758        24.0    8   19  ...  1.0   2.0    7   17   Forward
4   2011-12  1610612762        35.0   81  174  ...  2.0  21.0   78  217     Guard

[5 rows x 22 columns]
Training label: 0    934.0
1    147.0
2    753.0
3     43.0
4    796.0
Name: MIN, dtype: float64


In [70]:
# Display rows with any missing values in the training data
rows_with_missing_values = train_data[train_data.isnull().any(axis=1)]

print("Rows with missing values:")
print(rows_with_missing_values)

Rows with missing values:
      SEASON_ID     TEAM_ID  PLAYER_AGE     MIN  ...    BLK    TOV   PF   PTS
2       1964-65  1610612765        28.0   753.0  ...    NaN    NaN  136   208
6       1977-78  1610612762        25.0  2119.0  ...  129.0  225.0  293   833
15      1978-79  1610612765        22.0  2560.0  ...  201.0  141.0  254  1056
17      1946-47  1610610035        23.0     NaN  ...    NaN    NaN  119   152
25      1973-74  1610612739        22.0  1862.0  ...   35.0    NaN  192   500
...         ...         ...         ...     ...  ...    ...    ...  ...   ...
14979   1976-77  1610612755        25.0  2037.0  ...   15.0    NaN  174  1062
14981   1963-64           0        24.0   147.0  ...    NaN    NaN   35    26
14986   1962-63  1610612765        27.0  2961.0  ...    NaN    NaN  234  1547
14992   1973-74  1610612745        30.0    38.0  ...    0.0    NaN    4    14
14996   1970-71  1610612747        36.0    57.0  ...    NaN    NaN    6    20

[3048 rows x 22 columns]


In [71]:
# Fill the missing values
train_feature = train_data.drop(columns=['MIN'])
train_label = train_data['MIN']

categorical_cols = train_feature.select_dtypes(include=['object']).columns
numerical_cols = train_feature.select_dtypes(include=['int64', 'float64']).columns

train_feature[numerical_cols] = train_feature[numerical_cols].fillna(train_feature[numerical_cols].median())

for col in categorical_cols:
    train_feature[col] = train_feature[col].fillna(train_feature[col].mode()[0])

train_label = train_label.fillna(train_label.median())
print(train_feature.info())
print(train_label.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEASON_ID   15000 non-null  object 
 1   TEAM_ID     15000 non-null  int64  
 2   PLAYER_AGE  15000 non-null  float64
 3   FGM         15000 non-null  int64  
 4   FGA         15000 non-null  int64  
 5   FG_PCT      15000 non-null  float64
 6   FG3M        15000 non-null  float64
 7   FG3A        15000 non-null  float64
 8   FG3_PCT     15000 non-null  float64
 9   FTM         15000 non-null  int64  
 10  FTA         15000 non-null  int64  
 11  FT_PCT      15000 non-null  float64
 12  OREB        15000 non-null  float64
 13  DREB        15000 non-null  float64
 14  REB         15000 non-null  float64
 15  AST         15000 non-null  int64  
 16  STL         15000 non-null  float64
 17  BLK         15000 non-null  float64
 18  TOV         15000 non-null  float64
 19  PF          15000 non-nul

In [72]:
# Data transformation
train_feature['SEASON_ID'] = train_feature['SEASON_ID'].str[:4].astype(int)

print("Number of rows in train_feature:", len(train_feature))
print("Number of rows in train_label:", len(train_label))

Number of rows in train_feature: 15000
Number of rows in train_label: 15000


In [73]:
# Split the data
feature_train, feature_val, label_train, label_val = train_test_split(
    train_feature, 
    train_label, 
    test_size=0.2,
    random_state=10000
)

In [74]:
# 1. Random forest
rf_model = RandomForestRegressor(random_state=50)
rf_model.fit(feature_train, label_train)

rf_importances = rf_model.feature_importances_
rf_importance_df = pd.DataFrame({
    'Feature': feature_train.columns,
    'Importance': rf_importances
}).sort_values(by='Importance', ascending=False)

print("rf feature importances:")
print(rf_importance_df)

rf feature importances:
       Feature  Importance
3          FGM    0.506621
20         PTS    0.355197
4          FGA    0.035802
14         REB    0.030559
19          PF    0.015918
0    SEASON_ID    0.011719
15         AST    0.010291
16         STL    0.010017
13        DREB    0.008245
5       FG_PCT    0.001942
11      FT_PCT    0.001822
7         FG3A    0.001506
12        OREB    0.001500
18         TOV    0.001419
2   PLAYER_AGE    0.001382
10         FTA    0.001372
9          FTM    0.001277
17         BLK    0.001195
6         FG3M    0.001149
8      FG3_PCT    0.000957
1      TEAM_ID    0.000109


In [75]:
# Exclude low importance features and re-train
rf_low_importance_features = [feature for feature, importance in zip(feature_train.columns, rf_importances) if importance < 0.01]

print("features with low importance:", rf_low_importance_features)

rf_features_train = feature_train.drop(columns=rf_low_importance_features)
rf_features_val = feature_val.drop(columns=rf_low_importance_features)

rf = RandomForestRegressor(random_state=50)
rf.fit(rf_features_train, label_train)

label_pred_rf = rf.predict(rf_features_val)
mse_rf = mean_squared_error(label_val, label_pred_rf)
print("1. rf mse:", mse_rf)

features with low importance: ['TEAM_ID', 'PLAYER_AGE', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'BLK', 'TOV']
1. rf mse: 26960.0801871759


In [76]:
# Hyperparameter tuning for rf
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(
    RandomForestRegressor(random_state=50),
    rf_param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=-1
)

rf_grid_search.fit(rf_features_train, label_train)
print("best params for rf:", rf_grid_search.best_params_)

best params for rf: {'max_depth': 30, 'min_samples_leaf': 1, 'n_estimators': 300}


In [77]:
best_rf = rf_grid_search.best_estimator_

label_pred_rf_best = best_rf.predict(rf_features_val)
mse_best_rf = mean_squared_error(label_val, label_pred_rf_best)

print("1+. best rf mse:", mse_best_rf)

1+. best rf mse: 27051.00446684558


In [78]:
# 2. XGBoost
xgb_model = XGBRegressor(random_state=50)
xgb_model.fit(feature_train, label_train)

xgb_importances = xgb_model.feature_importances_
xgb_importance_df = pd.DataFrame({
    'Feature': feature_train.columns,
    'Importance': xgb_importances
}).sort_values(by='Importance', ascending=False)

print("xgb feature importances:")
print(xgb_importance_df)

xgb feature importances:
       Feature  Importance
20         PTS    0.562220
3          FGM    0.316480
4          FGA    0.035622
14         REB    0.025839
19          PF    0.019301
15         AST    0.007490
13        DREB    0.007338
16         STL    0.006195
0    SEASON_ID    0.005260
7         FG3A    0.002866
6         FG3M    0.001777
18         TOV    0.001209
12        OREB    0.001148
5       FG_PCT    0.001022
9          FTM    0.001021
17         BLK    0.000971
10         FTA    0.000960
11      FT_PCT    0.000949
8      FG3_PCT    0.000821
2   PLAYER_AGE    0.000761
1      TEAM_ID    0.000752


In [79]:
xgb_low_importance_features = [feature for feature, importance in zip(feature_train.columns, xgb_importances) if importance < 0.01]

print("features with low importance:", xgb_low_importance_features)

xgb_features_train = feature_train.drop(columns=xgb_low_importance_features)
xgb_features_val = feature_val.drop(columns=xgb_low_importance_features)

xgb = XGBRegressor(random_state=50)
xgb.fit(xgb_features_train, label_train)

label_pred_xgb = xgb.predict(xgb_features_val)
mse_xgb = mean_squared_error(label_val, label_pred_xgb)
print("2. xgb mse:", mse_xgb)

features with low importance: ['SEASON_ID', 'TEAM_ID', 'PLAYER_AGE', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TOV']
2. xgb mse: 50083.92965801424


Prediction on the test data

In [80]:
test_data_clean = test_data.drop(columns=['ID'])
test_data_clean['SEASON_ID'] = test_data_clean['SEASON_ID'].str[:4].astype(int)

test_pred_rf = rf.predict(test_data_clean.drop(columns=rf_low_importance_features))
test_pred_best_rf =  best_rf.predict(test_data_clean.drop(columns=rf_low_importance_features))
test_pred_xgb = xgb.predict(test_data_clean.drop(columns=xgb_low_importance_features))

result_rf = pd.DataFrame({
    'ID': test_data['ID'],
    'MIN': test_pred_rf
})

result_best_rf = pd.DataFrame({
    'ID': test_data['ID'],
    'MIN': test_pred_best_rf
})

result_xgb = pd.DataFrame({
    'ID': test_data['ID'],
    'MIN': test_pred_xgb
})

result_rf.to_csv('rf0.csv', index=False)
result_best_rf.to_csv('rf4.csv', index=False)
result_xgb.to_csv('xgb.csv', index=False)
print("Predictions saved")

Predictions saved
